In [40]:
import pymysql
import sys
import sqlalchemy as sa
import re

!{sys.executable} -m pip install SQLAlchemy 
!{sys.executable} -m pip install pymysql 

In [41]:
def intTo8mer(mernum):
    '''This function takes in the integer values that uniprobe stores to represent 8mers, and converts them back into 8mers.'''
   
    kmer = ''
    
    while(mernum>0):    
        nVal = mernum%4
        if nVal == 0:
            n = 'A'
        elif nVal == 1:
            n = 'C'
        elif nVal == 2:
            n = 'G'
        elif nVal == 3:
            n = 'T'
        else:
            sys.exit("Invalid 8mer")
        
        kmer = n + kmer
        mernum = (mernum - nVal)/4
    
    while(len(kmer) < 8):
        kmer = 'A' + kmer
      
    return kmer

In [42]:
engine = sa.create_engine('mysql+pymysql://'<DB_USER>':'<DB_PASS>'@'<DB_HOST>'/'<DB_NAME>)

In [43]:
conn = engine.connect()
meta = sa.MetaData()
meta.reflect(engine)

In [44]:
for table in meta.sorted_tables:
    print (table)

clone_to_protseq
contig_8mers_0
contig_8mers_1
contig_8mers_10
contig_8mers_11
contig_8mers_12
contig_8mers_13
contig_8mers_14
contig_8mers_15
contig_8mers_16
contig_8mers_17
contig_8mers_18
contig_8mers_19
contig_8mers_2
contig_8mers_20
contig_8mers_21
contig_8mers_22
contig_8mers_23
contig_8mers_25
contig_8mers_26
contig_8mers_3
contig_8mers_30
contig_8mers_32
contig_8mers_33
contig_8mers_34
contig_8mers_35
contig_8mers_36
contig_8mers_37
contig_8mers_4
contig_8mers_5
contig_8mers_6
contig_8mers_7
contig_8mers_8
contig_8mers_9
dbds
gene_id_complex
gene_ids
gene_ids_public
genomic_info_complex
licensed_files
log_downloads
log_queries
log_visitors
motif_map
mouse_pbm_pub
myguests
page_mod_date
pfam_beta
protein_complex
publication_ids
species_table
tablom
tf_heterodimers
tf_homodimers
tf_mod_date
trans_factors
users
version_to_amadid


In [45]:
ulist = []
unidict = {}
for table in meta.sorted_tables:
    if re.match('contig',str(table)):
        s = table.select(table)
        a = conn.execute(s)
        for result in a:
            unidict[result[0]] = {}
            ulist.append([result[0],result[1],result[2]])
for u in unidict:
    unidict[u]['8mers'] = {}
for result in ulist:
    unidict[result[0]]['8mers'][intTo8mer(result[1])]=result[2]    

In [47]:
count = 0
for i in unidict:
    count+=1
print(count)

776


In [48]:
unidict2 = unidict
genes = meta.tables['trans_factors']

query = genes.select(genes)

results = conn.execute(query)

pid_info = {}

for result in results: 
    if result['gene_id'] not in pid_info:
        pid_info[result['gene_id']] = [result['gene_name'], result['species'], result['publication_id'], [result['clone']]]
    else:
        pid_info[result['gene_id']][3].append(result['clone'])

for k,v in unidict2.items():
    if k in pid_info:
        unidict2[k]['protein'] = pid_info[k][0]
        unidict2[k]['species'] = pid_info[k][1]
        unidict2[k]['pubid'] = pid_info[k][2]
        unidict2[k]['clone_ids'] = pid_info[k][3]

In [49]:
genes2 = meta.tables['gene_ids_public']

query = genes2.select(genes2)

results = conn.execute(query)

pid_info2 = {}
for result in results:
    if not result['gene_name']:
        pid_info2[result['gene_id']] = [result['gene_id_name'], result['species'], result['publication_id']]
    else:
        pid_info2[result['gene_id']] = [result['gene_name'], result['species'], result['publication_id']]


for k,tf in unidict2.items():
    if 'protein' not in tf:
        print (k)
        unidict2[k]['protein'] = pid_info2[k][0]
        unidict2[k]['species'] = pid_info2[k][1]
        unidict2[k]['pubid'] = pid_info2[k][2]


427
494
493
492
491
490
489
488
487
486
536
535
551
550
549
360
362
366
372
374
375
376
377
380
385
808
807
806
805
804
803
802
801
800
799
798
797
796
794
793
792
791
790
789
788
787
786
785
784
783
782
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
1345
761
760
759
758
757
756
755
754
753
752
751
750
749
748
747
746
745
744
743
742
741
740
739
738
737
736
735
734
733
732
731
727
726
725
724
723
722
721
720
719
718
717
716
715
714
713
712
711
710
709
708
707
706
705
704
703
702
701
700
695
694
693
692
691
690
689
688
687
686
685
684
683
682
681
680
678
676
675
674
673
672
671
668
667
666
665
664
658
657
656
655
654
653
652
651
650
649
648
647
646
645
644
643
642
641
640
639
638
637
636
635
634
633
632
631
1423
1422
1421
1420
1419
1418
1417
1416
1406
1361
1356


In [51]:
print(len(unidict2))
count = 0
for k,tf in unidict2.items():
    if not tf['protein']:
        count+=1
print (count)

776
0


In [52]:
for tf in unidict2.values():
    print (tf.keys())

dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'protein', 'species', 'pubid', 'clone_ids'])
dict_keys(['8mers', 'prot

In [53]:
dbds = meta.tables['dbds']
query = dbds.select(dbds)
results = conn.execute(query)
dbdict = {}
for dbd in results:
    dbdict[dbd[0]] = dbd[1:]
for tf in unidict2.values():
    tf['dbd'] = []
    if tf['protein'] in dbdict:
        tf['dbd'].append(dbdict[tf['protein']][0])

In [60]:
count = 0
tot=0
for k,v in unidict2.items():
    tot+=1
    if 'family' not in v:
        count+=1
print(tot,count)

776 667


In [57]:
dbddict2 = {}
pfam = meta.tables['pfam_beta']
query = pfam.select(pfam)
results = conn.execute(query)
for result in results:
    protein_name = result['seq_id'].split('_')[1]
    if protein_name in dbddict2:
        dbddict2[protein_name].append(result['domain_seq'])
    else:
        dbddict2[protein_name] = [result['pfam_id'], result['domain_seq']]
for tf in unidict2.values():
    if tf['protein'] in dbddict2:
        tf['dbd'].append(dbddict2[tf['protein']][1:])
        tf['family'] = dbddict2[tf['protein']][0]

In [61]:
dbddict3 = {}
cloneseq = meta.tables['clone_to_protseq']
query = cloneseq.select(cloneseq)
results = conn.execute(query)
dbdlist = []
for result in results:
    dbdlist.append(result)
        

In [66]:
for result in dbdlist:
    print(result['clone'])
for tf in unidict2.values():
    if 'clone_ids' in tf:
        for result in dbdlist:
            if result['clone'] in tf['clone_ids']:
                if result['mutation']:
                    tf['dbd'].append({result['mutation'] : result['prot_sequence']})
                else:
                    tf['dbd'].append(result['prot_sequence'])

900
913
920
935
952
953
997
1030
1040
1063
1275
1286
1719
1720
1722
1728
1731
1732
1738
1741
1742
1744
1765
2205
2218
2226
2238
2240
2242
2245
2246
2247
2272
2273
2274
2277
2279
2281
2284
2302
2307
2310
2318
2325
2327
2335
2342
2343
2356
2364
2367
2368
2385
2590
2622
2627
2630
2641
2645
2668
2674
2779
2823
2824
2860
2877
2883
3031
3074
3077
3079
3081
3082
3122
3123
3124
3126
3127
3203
3206
3214
3235
3413
3415
3418
3419
3420
3422
3423
3424
3425
3426
3428
3429
3430
3431
3432
3433
3436
3437
3438
3439
3440
3441
3442
3443
3445
3446
3447
3448
3449
3451
3479
3480
3481
3484
3485
3486
3487
3488
3490
3491
3492
3493
3494
3496
3497
3498
3499
3500
3718
3719
3733
3748
3750
3773
3780
3783
3815
3818
3819
3838
3845
3856
3868
3869
3873
3947
3948
3949
3952
3953
3954
3956
3963
3970
3986
3989
3990
4251
2623
3072
3110
3483
2312
2644
2791
2825
3435
4272
2350
3219
895
905
911
921
925
950
961
972
973
974
982
991
1011
1022
1024
1034
1035
1047
1048
1051
1054
1274
1717
1718
1727
1733
1745
1753
1757
2190
2192
2241

In [70]:
count = 0
tot=0
for k,v in unidict2.items():
    tot+=1
    if 'clone_ids' in v:
        print (v['dbd'])


['...*']
['RSCLLCRRRKQRCDHKLPSCTACLKAGIKCVQPSKYSSS']
['MAAKRGLAKQKSRVTKACDRCHRKKIKCNSKKPCFGCIGSQSKCTYRNQFREPIEAFFNYTGSLSNDLDNAKCSIAKLKAQLPPSAPASLQKGLANICTELEKIQPQLYLNLDSKEISSYGGLKSIETEIIGKQSKSLNRFSNAFESNTAQNVSMYFGVYSPLLYFASTGISWITKKLISCSNDRETRETIYLFLKFLDASSASHAGPKVTSISPLEYYSKLNGLSCGNDVLIQHIMSNISNEIKGNTNINQTIKFNKPTDWFMYGVQLMEQHHKALDRKSSKKLLPLKYFLEQDELIFCLCLEYFERSLFSTMYDLTILKGLVSLMKHRYWIDDPFVLGRIISTMSRRSLDAGLNRWEYYIGQDEDTAEEYRKLWWDCYWWDRWYSLVTGKQPLIPHEMTSCLFPKDVVGLGVDDSMDCFTLINLVELDPSKFDICISFGYILLTKIITAVFSGLLYNRHFTDYRLFATPNAKDLNGTARQLMVEFSKICKIFQCIQDKLIPFLKQYSENSNVFELYTHFGFAKVCCFQGMESLILRIQNLLQERERIELDSCVKDIRLQTFEASVDILTDVLKHEDTFYIFRCSWFIYAILMNITLNFIETPRRNSICYLSLMCRMIASYNDLFVSSGNVNFKGNNAFSKKLENGTAVSFILTRICCQMYTRSQKMAKESLFCELKKYGQACSDAGQAALDIECIWYRNIIGEHKESSFRKEILSILDREMGDLVNNRVIGVQGKNQEGACYEKLSPSSTSVSVGMDFCSLENFVTAESLPDLLNLFWEDTEFGITKENLGE*']
['MSHHVPNLYGTPIRDPHERKRNSASMGEVNQSVSSRNCERGSEKGTKQRKKASHACDQCRRKRIKCRFDKHTGVCQGCLEVGEKCQFIRVPLKRGPAKKRGSVVSIEKFSSDNDPLQYRPRTHSYPMNSGNNYLPSLARNSSFPS

In [72]:
import json
with open('uniprobe_pubmed_ids.json','r') as pm:
    pubmed = json.loads(pm.read())
for tf in unidict2.values():
    tf['pubmed'] = pubmed[str(tf['pubid'])]
for tf in unidict.values():
    del tf['pubid']

In [73]:
j = json.dumps(unidict2)
with open('uniprobe.json', 'w') as k:
    print(j, file=k)